In [1]:
### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

import helpers
NOTEBOOK_PATH = __vsc_ipynb_file__ # type: ignore
helpers.initialize(notebook_path=NOTEBOOK_PATH)

### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

✅ Added notebook directory to Python path:
   /workspaces/llmops-course/modules/cloud_mlflow/solutions
   You can now import modules from this directory
⚠️ Warning: Failed to set up notebook path
🔄 Initializing Course environment...
🔁 Autoreload enabled: modules will reload automatically when changed
🔕 Suppressed future deprecation warnings
📝 Logging configured
📊 Pandas display settings configured for better output
🔍 Looking for .env file at: /workspaces/llmops-course/.env
✅ Successfully loaded environment variables from /workspaces/llmops-course/.env
📋 Loaded variables: GEMINI_API_KEY=****sbqo, AZURE_MLFLOW_TRACKING_URI=azure...
⚙️ Disabled MLflow system metrics logging
📔 Disabled MLflow notebook display (avoids VSCode bugs)

🎉 All systems go! Your Course environment is ready for learning!


In [2]:
import mlflow
import os

mlflow.set_tracking_uri(os.getenv("AZURE_MLFLOW_TRACKING_URI"))
mlflow.set_experiment("Summarizer Extension")


/home/vscode/.local/lib/python3.12/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
Parsing tracking uri /mlflow/v1.0/subscriptions/c5e6f73f-08ee-40cd-b716-20e405409b61/resourceGroups/llmops-prep/providers/Microsoft.MachineLearningServices/workspaces/AzureMLLLMOps
Tracking uri /mlflow/v1.0/subscriptions/c5e6f73f-08ee-40cd-b716-20e405409b61/resourceGroups/llmops-prep/providers/Microsoft.MachineLearningServices/workspaces/AzureMLLLMOps has sub id c5e6f73f-08ee-40cd-b716-20e405409b61, resource group llmops-prep, and workspace AzureMLLLMOps
Fetching cloud metadata from known urls


Cloud was fetched from known metadataurls
Cloud metadata not found so falling back to AzureCloud as default
No environment configuration found.
ManagedIdentityCredential will use IMDS
Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential


<Experiment: artifact_location='', creation_time=1741096468503, experiment_id='5858af87-ed70-46bc-8eeb-f7565484a978', last_update_time=None, lifecycle_stage='active', name='Summarizer Extension', tags={}>

In [3]:

import glob
import pandas as pd
from pathlib import Path

# Recursively find all HTML files in assets directory
html_files = glob.glob("assets/articles_short/*.html")

# Create list to store file data
file_data = []

# Read each HTML file
for file_path in html_files:
    content = open(file_path, 'r', encoding='utf-8').read()
    file_data.append({
        'file_name': Path(file_path).name,
        'text': content
    })

# Create DataFrame
eval_df = pd.DataFrame(file_data)


path = 'assets/articles_full_length/ai_relationships.html'
text = open(path).read()

input_example = pd.DataFrame({'text': [text]})


In [4]:
from modules.cloud_mlflow.solutions.textsummarizer import SerializableArticleSummarizerModel

prompt = "Summarize the article in 2 sentences: {text}"
max_output_tokens = 200
simple_model = SerializableArticleSummarizerModel(prompt=prompt, max_output_tokens=max_output_tokens)
eval_df["summaries"] = simple_model.predict(eval_df)

eval_df

file_name  \
0  the_end_of_internet_search.html   
1         electricity_demants.html   
2            ai_relationships.html   
3            prompt_injection.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [5]:
from mlflow.metrics import MetricValue, make_metric
from mlflow.metrics.base import standard_aggregations
from modules.cloud_mlflow.solutions.textsummarizer import REFUSAL_TEXT

def model_refused_to_respond(predictions, targets):
    scores = []
    for prediction in predictions:
        if prediction == REFUSAL_TEXT:
            scores.append(1)
        else:
            scores.append(0)

    return MetricValue(
        scores=scores,
        aggregate_results=standard_aggregations(scores),
    )


# Create an EvaluationMetric object.
refusal_rate_metric = make_metric(
    eval_fn=model_refused_to_respond, greater_is_better=False, name="refusal_rate"
)

path = Path('assets/articles_full_length/ai_relationships.html')
text = path.read_text()

input_example = pd.DataFrame({'text': [text]})


In [ ]:
with mlflow.start_run(run_name="Simple Model") as run:
    eval_result = mlflow.evaluate(
        data=eval_df,
        predictions="summaries",
        targets="text",
        model_type="text-summarization",
        extra_metrics=[
            refusal_rate_metric
        ],
    )

    mlflow.log_param("max_output_tokens", max_output_tokens)
    mlflow.log_param("prompt", prompt)

    mlflow.pyfunc.log_model(
        "article_summarizer",
        python_model=simple_model,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            simple_model.predict(input_example)
        ),
        code_paths=[Path(__vsc_ipynb_file__).parent],
        registered_model_name="article_summarizer",
    )


Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
2025/03/05 11:03:50 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
PyTorch version 2.6.0 available.
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
Device set to use cpu
Using default tokenizer.
Using default tokenizer.
Usin

In [28]:
from modules.cloud_mlflow.solutions.textsummarizer import secure_prompt
max_output_tokens = 200
secure_model = SerializableArticleSummarizerModel(prompt=secure_prompt, max_output_tokens=max_output_tokens)
eval_df["summaries"] = secure_model.predict(eval_df)


In [51]:
with mlflow.start_run(run_name="Secure Model") as run:
    eval_result = mlflow.evaluate(
        data=eval_df,
        predictions="summaries",
        targets="text",
        model_type="text-summarization",
        extra_metrics=[
            refusal_rate_metric
        ],
    )

    mlflow.log_param("max_output_tokens", max_output_tokens)
    mlflow.log_param("prompt", prompt)

    model_info = mlflow.pyfunc.log_model(
        "article_summarizer",
        python_model=secure_model,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            secure_model.predict(input_example)
        ),
        code_path=[Path(__vsc_ipynb_file__).parent],
        registered_model_name="article_summarizer",
    )


Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
2025/03/05 10:57:35 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using de

🏃 View run Secure Model at: https://westeurope.api.azureml.ms/mlflow/v2.0/subscriptions/c5e6f73f-08ee-40cd-b716-20e405409b61/resourceGroups/llmops-prep/providers/Microsoft.MachineLearningServices/workspaces/AzureMLLLMOps/#/experiments/5858af87-ed70-46bc-8eeb-f7565484a978/runs/ca996094-152c-4710-93cd-ee7892b4d34f
🧪 View experiment at: https://westeurope.api.azureml.ms/mlflow/v2.0/subscriptions/c5e6f73f-08ee-40cd-b716-20e405409b61/resourceGroups/llmops-prep/providers/Microsoft.MachineLearningServices/workspaces/AzureMLLLMOps/#/experiments/5858af87-ed70-46bc-8eeb-f7565484a978


ChainedTokenCredential acquired a token from AzureCliCredential


In [52]:
# Remove production stage tag from all versions
client = mlflow.tracking.MlflowClient()
for mv in client.search_model_versions(f"name='article_summarizer'"):
    try:
        client.delete_model_version_tag(mv.name, mv.version, "stage")
    except:
        pass  # Tag may not exist

# Add production stage tag to current version
client.set_model_version_tag(
    name="article_summarizer",
    version=model_info.registered_model_version,
    key="stage",
    value="production"
)
model_info.registered_model_version

Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential
Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to th

'6'

In [53]:
# Get the latest production model
client = mlflow.tracking.MlflowClient()
model_versions = client.search_model_versions("name='article_summarizer'")
sorted_versions = sorted(model_versions, key=lambda x: int(x.version), reverse=True)

production_model = None
for version in sorted_versions:
    if "stage" in version.tags and version.tags["stage"] == "production":
        print(f'Production model found: {version.version}')
        print(f'Production Model URI:\n========\nmodels:/article_summarizer/{version.version}\n========')
        break
else:
    raise Exception("No production model found")


Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.8 (Linux-6.10.14-linuxkit-aarch64-with-glibc2.31)'
No body was attached to the request
ChainedTokenCredential acquired a token from AzureCliCredential


Production model found: 6
Production Model URI:
models:/article_summarizer/6
